In [ ]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
#import data 
on_df = spark.sql("select * from df")
#convert data types to integer or as required 
on_df=on_df.withColumn("SensorOcc107",col("SensorOcc107").cast("integer"))
# converting datetime type
sensors_on=sensors_on.withColumn("datetype",to_date(col("date_time"),"yyyy-MM-dd"))
#handling missing data by imputing = for example putting '0' as substitute 
on_df=on_df.na.fill(value=0)

In [ ]:
#creating new column by adding several columns 
on_df=on_df.withColumn("ts",col("SensorOcc000")+col("SensorOcc100")+col("SensorOcc101")+col("SensorOcc102")+col("SensorOcc103")+col("SensorOcc104")+col("SensorOcc105")+col("SensorOcc106")+col("SensorOcc107"))
#creating subset by selecting varibles of interest
on_df=on_df.select("ts", "identifier", "date_time")
#import Metadata for identifiers with specific selection criteria 
metadata = spark.sql("select * from metadata")
new_meta=metadata[(metadata["Country"]=="CA") & (metadata["ProvinceState"]=="BC") & (metadata['Number_of_Occupants']==2) & (metadata['Number_of_Remote_Sensors']>6)]
new_meta=new_meta.toPandas()

In [ ]:
#time zone handling 
import pytz
eastern = pytz.timezone('MST')
on_df=on_df.set_index("date_time")
on_df.index = on_df.index.tz_localize(pytz.utc).tz_convert(eastern)

In [ ]:
#data wrangling 
import pandas as pd
import numpy as np
sensorFrame = pd.DataFrame(df.filter(regex=(".*SensorOcc.*")))
sensorFrame = sensorFrame.fillna(value=np.nan)
sensorFrame = sensorFrame.fillna(0)
sensorFrame= sensorFrame.astype(int)
sensorFrame["totalsensors"] = sensorFrame.sum(axis=1)
ts=pd.DataFrame(sensorFrame["totalsensors"])
del sensorFrame
df2 = pd.merge(ts, df1,on=None,left_index=True, right_index=True)

In [ ]:
#frequnce count for one variable 
df.select(countDistinct("identifier")).show()

In [ ]:
#frequency count group by variable 
tableqc=on_df.groupBy("datetype").mean("ts")